In [ ]:
from data import en_chset
from data.gen import fonts, weights, get_mat, slice_img, save_chset_random
import itertools
import matplotlib.pyplot as plt
import random
import math
%matplotlib inline

f = open("data/k1001.txt")
ko_chset = f.read().splitlines()

all_chset = ko_chset + en_chset

def get_random_ch(chset=all_chset):
    return chset[random.randrange(0,len(chset))]

# Helper function to draw 3 X 3 plots
def draw_subplot(array, w, h):
    plt.figure(num=None, figsize=(3, 3), facecolor='w', edgecolor='k')
    for i in range(len(array)):
        plt.subplot(w,h,i+1)
        plt.xticks(())
        plt.yticks(())
        plt.imshow(array[i], interpolation="none", cmap=plt.get_cmap("gray"))

def show_example_all(ch, fonts, weights):
    sliced = []
    for font in fonts:
        print(font)
    for font, weight in itertools.product(fonts, weights):
        sliced.append(slice_img(get_mat(ch, font, weight)))
    draw_subplot(sliced, math.ceil(len(sliced)/2), 2)

In [ ]:
def show_example(ch):
    sliced = [slice_img(get_mat(ch)) for i in range(7)]
    draw_subplot(sliced, 1, 7)
    
def show_example_random():
    sliced = [slice_img(get_mat(get_random_ch())) for i in range(7)]
    draw_subplot(sliced, 1, 7)
    
for ch in ".", ";", "2", ")", "가", "낢", "·", "《", "》", "「", "」", "『", "』" :
    show_example(ch)
    
for i in range(3):
    show_example_random()

In [ ]:
show_example_all(".", fonts, weights)

In [ ]:
save_chset_random((en_chset, ko_chset), (1, 3), "data/161028.tgz", 300000)